<a href="https://colab.research.google.com/github/learn-programmers/programmers_kdt_II/blob/main/9%EC%A3%BC%EC%B0%A8_PySpark_%EA%B8%B0%EB%B3%B8_2%EC%9D%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [5]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 MB 25.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 11.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612229 sha256=0984c47b1ab15c78e69eb3035ad630491c8b2b9098f561260ae5dbcd6126580c
  Stored in directory: /Users/jykim/Library/Caches/pip/wheels/19/b0/c8/6cb894117070e130fc44352c2a13f15b6c27e440d04a84fb48
Successfully built pyspark


In [7]:
!ls -tl

total 55856
-rw-r--r--  1 jykim  staff     45706 Nov 11 16:04 3일차_SQL_실습.ipynb
-rw-r--r--  1 jykim  staff  21537493 Nov 11 16:04 2일차_SQL_실습.ipynb
-rw-r--r--  1 jykim  staff    116585 Nov 11 16:04 4일차_실습.ipynb
-rw-r--r--  1 jykim  staff   1082847 Nov 11 16:02 5일차_실습.ipynb
-rw-r--r--  1 jykim  staff   2656818 Nov 11 15:49 SQL_JOIN.ipynb
-rw-r--r--  1 jykim  staff    449040 Nov 11 15:49 SQL_last.ipynb
-rw-r--r--  1 jykim  staff    521170 Nov  9 23:54 SQL_GROUP_BY_CTAS.ipynb
-rw-r--r--  1 jykim  staff         0 Nov  9 19:02 라이브세션-2.ipynb
-rw-r--r--  1 jykim  staff   1930247 Nov  9 04:16 SQL_analysis.ipynb
-rw-r--r--  1 jykim  staff     53662 Nov  7 15:08 11주차_SQL_실습_3일차_1.ipynb
-rw-r--r--  1 jykim  staff     65835 Nov  7 15:08 11주차_PySpark_기본_5일차_1.ipynb
-rw-r--r--  1 jykim  staff     30077 Nov  7 15:08 11주차_PySpark_기본_4일차_2.ipynb
-rw-r--r--  1 jykim  staff     32231 Nov  7 15:08 11주차_PySpark_기본_4일차_1.ipynb
-rw-r--r--  1 jykim  staff   

In [8]:
!ls -tl sample_data

ls: sample_data: No such file or directory


**Spark Session:** SparkSession은 Spark 2.0부터 엔트리 포인트로 사용된다. 그 이전에는 SparkContext가 사용되었다. SparkSession을 이용해 RDD, 데이터 프레임등을 만든다. SparkSession은 SparkSession.builder를 호출하여 생성하며 다양한 함수들을 통해 세부 설정이 가능하다

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

23/11/11 16:08:50 WARN Utils: Your hostname, KimJY.local resolves to a loopback address: 127.0.0.1; using 192.168.0.23 instead (on interface en0)
23/11/11 16:08:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Exception in thread "main" java.lang.ExceptionInInitializerError
	at org.apache.spark.unsafe.array.ByteArrayMethods.<clinit>(ByteArrayMethods.java:54)
	at org.apache.spark.internal.config.package$.<init>(package.scala:1006)
	at org.apache.spark.internal.config.package$.<clinit>(package.scala)
	at org.apache.spark.deploy.SparkSubmitArguments.$anonfun$loadEnvironmentArguments$3(SparkSubmitArguments.scala:157)
	at scala.Option.orElse(Option.scala:447)
	at org.apache.spark.deploy.SparkSubmitArguments.loadEnvironmentArguments(SparkSubmitArguments.scala:157)
	at org.apache.spark.deploy.SparkSubmitArguments.<init>(SparkSubmitArguments.scala:115)
	at org.apache.spark.deploy.SparkSubmit$$anon$2$$anon$3.<init>(SparkSubmit.scala:990)
	at org.apache.spark.deploy.Spark

Exception: Java gateway process exited before sending its port number

In [2]:
spark

NameError: name 'spark' is not defined

**Python 객체를 RDD로 변환해보기**

**1> Python 리스트 생성**

In [ ]:
name_list_json = [ '{"name": "keeyong"}', '{"name": "benjamin"}', '{"name": "claire"}' ]

In [ ]:
for n in name_list_json:
  print(n)

{"name": "keeyong"}
{"name": "benjamin"}
{"name": "claire"}


In [ ]:
import json

for n in name_list_json:
  jn = json.loads(n)
  print(jn["name"])

keeyong
benjamin
claire


**2> 파이썬 리스트를 RDD로 변환. RDD로 변환되는 순간 Spark 클러스터의 서버들에 데이터가 나눠 저장됨 (파티션). 또한 Lazy Execution이 된다는 점 기억**

In [ ]:
rdd = spark.sparkContext.parallelize(name_list_json)

In [ ]:
rdd

ParallelCollectionRDD[4] at readRDDFromFile at PythonRDD.scala:262

In [ ]:
rdd.count()

3

In [ ]:
parsed_rdd = rdd.map(lambda el:json.loads(el))

In [ ]:
parsed_rdd

PythonRDD[6] at RDD at PythonRDD.scala:53

In [ ]:
parsed_rdd.collect()

[{'name': 'keeyong'}, {'name': 'benjamin'}, {'name': 'claire'}]

In [ ]:
parsed_name_rdd = rdd.map(lambda el:json.loads(el)["name"])

In [ ]:
parsed_name_rdd.collect()

['keeyong', 'benjamin', 'claire']

**파이썬 리스트를 데이터프레임으로 변환하기**

In [ ]:
from pyspark.sql.types import StringType

df = spark.createDataFrame(name_list_json, StringType())

In [ ]:
df.count()

3

In [ ]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [ ]:
df.select('*').collect()

[Row(value='{"name": "keeyong"}'),
 Row(value='{"name": "benjamin"}'),
 Row(value='{"name": "claire"}')]

In [ ]:
df.select('value').collect()

[Row(value='{"name": "keeyong"}'),
 Row(value='{"name": "benjamin"}'),
 Row(value='{"name": "claire"}')]

In [ ]:
from pyspark.sql import Row

row = Row("name") # Or some other column name
df_name = parsed_name_rdd.map(row).toDF()

In [ ]:
df_name.printSchema()

root
 |-- name: string (nullable = true)



In [ ]:
df_name.select('name').collect()

[Row(name='keeyong'), Row(name='benjamin'), Row(name='claire')]